# ArcGIS Online Web Application Audit
Description: This will audit all of the web applications in an organization.
 
Beginning with: AGOWebApplicationAuditPythonAPI.py
 
Created on: 2/13/2018
 
Purpose: This will audit all of the web applications in an organization,
    and provide information about the web maps that are referenced by the application. 
 
Authored by: Brandon Longenberger
 
Previous Production Date: 2/13/18     Production Date: 9/3/18

# Connect to ArcGIS Online

In [ ]:
# import
from arcgis.gis import GIS
from arcgis.gis.server import Server
import getpass
from IPython.display import display
from arcgis.mapping import WebMap
import json
import requests

# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass('Password: ')

# Connection 
gis = GIS(Organization, User, Password)
gis
WebAppAudit = gis.content.search(query="", item_type="Web Mapping Application", max_items = 10000000)
WebMapAudit = gis.content.search(query="", item_type="Web Map", max_items = 10000000)
MapList = []

# Audit Section

In [ ]:
# Functions
def WriteMapInfo(MapID):
    global df
    MapItemList = gis.content.search(query='id:' + str(MapID))
    for MapItem in MapItemList:
        
        AddToMapList(MapID)
        
        if Item.access == MapItem.access:
            EqualSharing = "Yes"
        else:
            EqualSharing = "No"
        df = df.append({
        'AppTitle':str(Item.title.replace(",", "")),
        'AppID':str(Item.id),
        'AppItemType':str(Item.type),
        'AppOwner':str(Item.owner),
        'AppAccess':str(Item.access),
        'MapTitle':str(MapItem.title.replace(",", "")),
        'MapID':str(MapID),
        'ItemType':str(MapItem.type),
        'MapOwner':str(MapItem.owner),
        'MapAccess':str(MapItem.access),
        'EqualSharing':EqualSharing
        }, ignore_index=True)

def GetMedia(Story, Item):
    global df
    Media = Story.get("media")
    if Media != None:
        Type = Media.get("type")
        if Type == "webpage":
            Webpage = Media[Type]
            Url = Webpage["url"]
            df = df.append({
            'AppTitle':str(Item.title.replace(",", "")),
            'AppID':str(Item.id),
            'AppItemType':str(Item.type),
            'AppOwner':str(Item.owner),
            'Webpage':Media[Type],
            'WebPageUrl':str(Url),
            'MapID':"Web Page"
            }, ignore_index=True)
        if Type == "webmap":
            Webmap = Media[Type]
            
            WriteMapInfo(Webmap["id"])

def LoopWebMaps(List):
    for MapID in List:
        WriteMapInfo(MapID)

def AddToMapList(MapID):
    global MapList
    if MapID not in MapList:
        MapList += [MapID]

# End Function Section

import pandas as pd
print("Pandas imported")
dataColumns = ['AppTitle','AppID','AppItemType','AppOwner','AppAccess','WebPageUrl','MapTitle','MapID',
               'ItemType','MapOwner','MapAccess','EqualSharing']
df = pd.DataFrame(columns = dataColumns)

for Item in WebAppAudit:
    ItemData = Item.get_data(try_json=True)
    if ItemData != None:
        Map = ItemData.get("map")
        if Map != None:
            MapItemID = Map.get("itemId")
            WriteMapInfo(MapItemID)
        Values = ItemData.get("values")
        if Values != None:
            WebMap = Values.get("webmap")
            if WebMap != None:
                if isinstance(WebMap, str):
                    Find = WebMap.find(",")
                    if Find == -1:
                        WriteMapInfo(WebMap)
                    else:
                        Split = WebMap.split(",")
                        LoopWebMaps(Split)
                elif isinstance(WebMap, list):
                    LoopWebMaps(WebMap)
            Story = Values.get("story")
            if Story != None:
                Sections = Story.get("sections")
                if Sections != None:
                    for Section in Sections:
                        GetMedia(Section, Item)
                Entries = Story.get("entries")
                if Entries != None:
                    for Entry in Entries:
                        GetMedia(Entry, Item)
    else:
        df = df.append({
        'AppTitle':str(Item.title.replace(",", "")),
        'AppID':str(Item.id),
        'AppItemType':str(Item.type),
        'AppOwner':str(Item.owner)
        }, ignore_index=True)

df.drop_duplicates(subset=dataColumns,inplace=True)
df

# Add Webmaps not in an App to dataframe

In [ ]:
for Map in WebMapAudit:
    if Map.id in MapList:
        EqualSharing = "Yes"
    else:
        EqualSharing = "No"
    df = df.append({
    'MapTitle':str(Map.title.replace(",", "")),
    'MapID':str(Map.id),
    'MapOwner':str(Map.owner),
    'MapAccess':str(Map.access),
    'EqualSharing':EqualSharing
    }, ignore_index=True)

print("Webmaps not in an App have been added to dataframe.")

# Write the Web App Audit Results to CSV

In [ ]:
from tkinter.filedialog import asksaveasfilename
filename = asksaveasfilename(defaultextension='.csv') # show an "Open" dialog box and return the path to the selected file
print(filename)
df.to_csv(filename, index=False)
print("Write data To CSV File Complete!")

# Experimenting with Pandas

In [ ]:
print (len(df))
mf = df
mf.duplicated(subset=None,keep=False)
print (len(mf))